In [1]:
import sys
!{sys.executable} -m pip install opencv-contrib-python --upgrade
!{sys.executable} -m pip install sympy

In [2]:
import cv2
import numpy as np

img1 = cv2.imread('./data/img_m1.jpg')
img2 = cv2.imread('./data/img_m2.jpg')
 
cv2.imshow("Img1",img1)
cv2.imshow("Img2",img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# convert image to RGB colour
img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# convert image to Grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [4]:
# create an instance of the Face Detection Cascade Classifier
detector = cv2.CascadeClassifier("./data/haarcascade_frontalface_default.xml")

# Detect faces using the haarcascade classifier on the "grayscale image"
f_img1 = detector.detectMultiScale(img1_gray)
f_img2 = detector.detectMultiScale(img2_gray)

face1 = img1[f_img1[0][1]:f_img1[0][1]+f_img1[0][3],f_img1[0][0]:f_img1[0][0]+f_img1[0][2]]
face2 = img2[f_img2[0][1]:f_img2[0][1]+f_img2[0][3],f_img2[0][0]:f_img2[0][0]+f_img2[0][2]]

cv2.imshow("Face1", face1)
cv2.imshow("Face2", face2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# create an instance of the Facial landmark Detector with the model
landmark_detector  = cv2.face.createFacemarkLBF()
landmark_detector.loadModel("./data/lbfmodel.yaml")

# Detect landmarks on "image_gray"
_, landmarks = landmark_detector.fit(img1_gray, f_img1)


h,w = img1.shape[:2]
l_img1 = np.concatenate((landmarks[0][0],np.array([[0,0],[w/2,0],[w-1,0], [0,h/2], [0,h-1], [w/2,h-1], [w-1,h/2],[w-1,h-1] ])))

np.savetxt("./data/img1_landmarks.txt",(l_img1).astype(int))

# Detect landmarks on "image_gray"
_, landmarks = landmark_detector.fit(img2_gray, f_img2)
h,w = img2.shape[:2]
l_img2 = np.concatenate((landmarks[0][0],np.array([[0,0],[w/2,0],[w-1,0], [0,h/2], [0,h-1], [w/2,h-1], [w-1,h/2],[w-1,h-1] ])))

np.savetxt("./data/img2_landmarks.txt",(l_img2).astype(int))


In [6]:
# Draw delaunay triangles
def rect_contains(rect, point):

    if point[0] < rect[0]:
        return False
    elif point[1] < rect[1]:
        return False
    elif point[0] > rect[2]:
        return False
    elif point[1] > rect[3]:
        return False
    return True

def draw_delaunay(img, subdiv, dic, flag=False ) :

    triangleList = subdiv.getTriangleList()
    size = img.shape
    r = (0, 0, size[1], size[0])
    triangle_map = []
    copy = img.copy()
    for t in triangleList :
        temp = []
        pt1 = (int(t[0]), int(t[1]))
        pt2 = (int(t[2]), int(t[3]))
        pt3 = (int(t[4]), int(t[5]))
        
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
            cv2.line(copy, pt1, pt2, (255,255,255), 1, cv2.LINE_AA , 0)
            cv2.line(copy, pt2, pt3, (255,255,255), 1, cv2.LINE_AA , 0)
            cv2.line(copy, pt3, pt1, (255,255,255), 1, cv2.LINE_AA , 0)
            if flag:
                triangle_map.append((dic[pt1],dic[pt2],dic[pt3]))
        
        
    return triangle_map, copy

In [7]:
# Rectangle to be used with Subdiv2D
size = img1.shape if (img1.shape[0]>img2.shape[0] and img1.shape[1]>img2.shape[1]) else img2.shape
rect = (0, 0, size[1], size[0])
alpha=0.5

# Create an instance of Subdiv2D
# IMPORTANT: REPLACE THIS WITH MY OWN DELAUNAY IMPLEMENTATION
subdiv = cv2.Subdiv2D(rect)
subdiv1 = cv2.Subdiv2D((0, 0, img1.shape[1], img1.shape[0]))
subdiv2 = cv2.Subdiv2D((0, 0, img2.shape[1], img2.shape[0]))

# Create an array of points.
points1 = []
points2 = []
points = []

# Read in the points from a text file
with open("./data/img1_landmarks.txt") as file :
    for line in file :
        x, y = line.split()
        points1.append((int(float(x)), int(float(y))))
        
# Read in the points from a text file
with open("./data/img2_landmarks.txt") as file :
    for line in file :
        x, y = line.split()
        points2.append((int(float(x)), int(float(y))))



# Compute weighted average point coordinates
for i in range(0, len(points1)):
    x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
    y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
    points.append((int(x),int(y)))

print(len(points1),len(points2),len(points))
dictionary = {x[0]:x[1] for x in list(zip(points, range(76)))}
# Insert points into subdiv
for p in points :
    subdiv.insert(p)
for p in points1 :
    subdiv1.insert(p)
for p in points2 :
    subdiv2.insert(p)

# Draw delaunay triangles
t_map, delauney = draw_delaunay( img2, subdiv, dictionary, True)
_, d1 = draw_delaunay( img1, subdiv1, dictionary)
_, d2 = draw_delaunay( img2, subdiv2, dictionary)

76 76 76


In [8]:
cv2.imshow("Delauney Img1", d1)
cv2.imshow("Delauney Img2", d2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
import sympy
from sympy.solvers.solveset import linsolve

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    # Given a pair of triangles, find the affine transform.
    a,b,c,d,e,f = sympy.symbols('a,b,c,d,e,f')
    equations = []
    for i in range(0, len(srcTri)):
        equations.append(dstTri[i][0]-a*srcTri[i][0]-b*srcTri[i][1]-c)
        equations.append(dstTri[i][1]-d*srcTri[i][0]-e*srcTri[i][1]-f)
    res = list(linsolve(equations, (a,b,c,d,e,f)))
    res = [float(i) for i in res[0]]
    warpMat = np.array([[res[0],res[1],res[2]],[res[3],res[4],res[5]]])
    
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []


    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

In [36]:
# Allocate space for final output
imgMorph = np.zeros(size, dtype = img1.dtype)
for i in range(len(t_map)) :
    x = int(t_map[i][0])
    y = int(t_map[i][1])
    z = int(t_map[i][2])

    t1 = [points1[x], points1[y], points1[z]]
    t2 = [points2[x], points2[y], points2[z]]
    t = [ points[x], points[y], points[z] ]

    # Morph one triangle at a time.
    morphTriangle(img1, img2, imgMorph, t1, t2, t, alpha)


# Show results
#cv2.imshow("Originals",np.hstack([img1, img2]))
cv2.imshow("Morph",imgMorph)
cv2.waitKey(0)
cv2.destroyAllWindows()

10251 


 9063 



7776 


 9063 



18894 


 15903 



13392 


 15903 



17112 


 21546 



30246 


 21546 



20349 


 16974 



17280 


 16974 



29106 


 37146 



45864 


 37146 



11340 


 11352 



11352 


 11352 



14454 


 15405 



16128 


 15405 



38916 


 30450 



21600 


 30450 



10653 


 8874 



7056 


 8874 



16896 


 15792 



14535 


 15792 



35712 


 30912 



26220 


 30912 



176841 


 181653 



187668 


 181653 



20382 


 18105 



16128 


 18105 



1320 


 1587 



1800 


 1587 



1200 


 1380 



1425 


 1380 



188400 


 190800 



194400 


 190800 



61701 


 51039 



40824 


 51039 



11220 


 10998 



10863 


 10998 



4455 


 3108 



2088 


 3108 



2835 


 1887 



1218 


 1887 



149040 


 161406 



173082 


 161406 



9240 


 8094 



7215 


 8094 



12561 


 10863 



9408 


 10863 



158340 


 171750 



184365 


 171750 



153738 


 166278 



178068 


 166278 



13542 
